In [69]:
!source activate twitterCOVID

In [2]:
from tweepy import OAuthHandler, AppAuthHandler
from tweepy import API, TweepError
from secrets import *
from textblob import TextBlob
import os
import json
import dataset
from datafreeze import freeze
import pandas as pd

# Consumer key authentication
auth = AppAuthHandler(consumer_key, consumer_secret) #using appauthhandler to retreive at a faster rate

# Access key authentication
# auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

## Problem: SearchAPI is limited in many ways (max 100 tweets per query is just one of them)

In [58]:
search = api.search(q = "covid", until = "2020-04-02", count = 100)

In [59]:
jsonres = [resi._json for resi in search]

In [60]:
results = pd.DataFrame(jsonres)

In [61]:
results.head()

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,extended_entities,quoted_status
0,Wed Apr 01 23:59:59 +0000 2020,1245501195554369536,1245501195554369536,RT @DonaldJTrumpJr: The same media wants to st...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,18732,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Wed Apr 01 23:59:59 +0000 2020,1245501195373948929,1245501195373948929,RT @Econ_Marshall: Whatever happened to the Se...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,55,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Wed Apr 01 23:59:59 +0000 2020,1245501195327848449,1245501195327848449,"If I had a dollar..."" COVID edition: If I had...",True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,11,77,False,False,en,NaN,NaN,NaN,NaN,NaN
3,Wed Apr 01 23:59:59 +0000 2020,1245501195315220480,1245501195315220480,@look_now Trotzdem können sie die Krankheit ve...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'de', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.245501e+18,1245500860320419843,...,0,0,False,False,de,NaN,NaN,NaN,NaN,NaN
4,Wed Apr 01 23:59:59 +0000 2020,1245501195273285632,1245501195273285632,RT @RexChapman: Sandra the orangutang started ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,101284,0,False,False,en,NaN,NaN,NaN,NaN,NaN


## Cracking Twitter APIs

From [link](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./). The seacrh method has some `max_id` and `since_id` arguments we can exploit to our advantage. These are like upper and lower boundaries for our tweets, and given tweets come sorted by post time, we can retrieve many many tweets going back up to 1 week.

In [28]:
import sys
import jsonpickle
import os

searchQuery = "covid"
maxTweets = 1000
tweetsPerQry = 100 # max set by twitter api
fName = "LastWeek.csv"

# the api search output SearchObject is an object that can be turned into a list of json str with the _json method
# the search object itself can be indexed like a list
sinceID = None # no lower limit (as far as necessary)
maxID = -1 #exploit negative indexing, last tweet
tweetCount = 0
# the ID is just an ordered number, If I tweet now and my tweet ID is 1 and
# your tweet comes next your tweet id will be 2


print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (maxID<=0):
                # if your maxID is negative (1st iteration) do this
                if (not sinceID):
                    new_tweets = api.search(q=searchQuery, count = tweetsPerQry)
                else: # if sinceID exists set sinceID as that minimum
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceID)
            else: # if maxID is positive ? 
                if (not sinceID): # if sinceID not defined don't set low boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1))
                else: # if sinceID exist set low and up boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1),
                                            since_id=sinceID)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json))
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            maxID = new_tweets[-1].id # set maxID to the latest ID in the previous search
        except TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break
print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

1
Downloaded 100 tweets
1246251926246711296
3
Downloaded 200 tweets
1246251915131805696
3
Downloaded 300 tweets
1246251904738504705
3
Downloaded 400 tweets
1246251895078993920
3
Downloaded 500 tweets
1246251883859251200
3
Downloaded 600 tweets
1246251872773603333
3
Downloaded 700 tweets
1246251863592316928
3
Downloaded 800 tweets
1246251854268416003
3
Downloaded 900 tweets
1246251842281058311
3
Downloaded 1000 tweets
1246251831988236289
Downloaded 1000 tweets, Saved to LastWeek.csv


In [87]:
import pandas as pd
import jsonpickle
tweets = []

for line in open("BackSearch/SomeKeywords.json", "r"):
    tweets.append(json.loads(line))
tweets = pd.DataFrame(tweets)
tweets


,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities,withheld_in_countries
0,Sat Apr 04 23:32:13 +0000 2020,1246581367900602368,1246581367900602368,@johncardillo So suddenly thousands would have...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.246477e+18,1246476545683861504,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Apr 04 23:32:13 +0000 2020,1246581367422373888,1246581367422373888,@citizen_steel There's a complicated web of ge...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.246580e+18,1246580340929544192,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Apr 04 23:32:12 +0000 2020,1246581365472141321,1246581365472141321,@kharioki More like covid-19 S,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.246569e+18,1246568947807399937,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,Sat Apr 04 23:32:12 +0000 2020,1246581365312614401,1246581365312614401,@ZenGuille Cheyenne bunker mention is for CoVi...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.246575e+18,1246574784680509440,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN
4,Sat Apr 04 23:32:12 +0000 2020,1246581364197068802,1246581364197068802,"COVID-19 precautions: Dubai Police patrols, ca...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,None,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Sat Apr 04 23:30:00 +0000 2020,1246580813514276864,1246580813514276864,"On Saturday, Schnuck's announced they will lim...",True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.socialnewsdesk.com"" rel=""n...",NaN,None,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN
996,Sat Apr 04 23:30:00 +0000 2020,1246580813413695494,1246580813413695494,"On Saturday, Schnuck's announced they will lim...",True,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.socialnewsdesk.com"" rel=""n...",NaN,None,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN
997,Sat Apr 04 23:30:00 +0000 2020,1246580813233303555,1246580813233303555,ATA - Fico: Growth in Covid-19 cases has remai...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,False,False,en,False,NaN,NaN,NaN,"{'media': [{'id': 1246580799371194368, 'id_str...",NaN
998,Sat Apr 04 23:30:00 +0000 2020,1246580812964716544,1246580812964716544,The future of disruptive FinTech post COVID-19...,False,"{'hashtags': [{'text': 'mobielonly', 'indices'...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://drumup.io"" rel=""nofollow"">dru...",NaN,None,...,0,False,False,en,False,NaN,NaN,NaN,NaN,NaN


In [84]:
res = json.loads("BackSearch/SomeKeywords.json")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)